In [1]:
from VGG.model import VGG
from Resnet.model import ResNet50, ResNet34
import torch
import torch.nn as nn  # Neural network layers
from data import load_data
from train import train_model
from test import test_model

learning_rate = 0.01
class_size = 5
batch_size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
train_loader, valid_loader, test_loader = load_data(batch_size, S=256, train_dir='./data/flower_photos/train', 
                                                                      val_dir='./data/flower_photos/validation', 
                                                                      test_dir='./data/flower_photos/test')

print(f"Train size: {len(train_loader)}, dim: {next(iter(train_loader))[0].shape}")
print(f"Valid size: {len(valid_loader)}, dim: {next(iter(valid_loader))[0].shape}")
print(f"Test size: {len(test_loader)}, dim: {next(iter(test_loader))[0].shape}")

Train size: 56, dim: torch.Size([64, 3, 224, 224])
Valid size: 2, dim: torch.Size([64, 3, 224, 224])
Test size: 1, dim: torch.Size([50, 3, 224, 224])


In [3]:
criterion = nn.CrossEntropyLoss()

model = ResNet34(class_size).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.0001, momentum=0.9)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64

In [4]:
train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs=9)
acc = test_model(model, test_loader, device)
print(f"Test Accuracy: {acc}\n")

Epoch [1/9], Avg Train Loss: 1.3935, Avg Valid Loss: 1.7075, Valid Accuracy: 0.5750
Epoch [2/9], Avg Train Loss: 1.1078, Avg Valid Loss: 1.1776, Valid Accuracy: 0.6125
Epoch [3/9], Avg Train Loss: 1.0786, Avg Valid Loss: 3.2598, Valid Accuracy: 0.5375
Epoch [4/9], Avg Train Loss: 0.9732, Avg Valid Loss: 0.7150, Valid Accuracy: 0.7250
Epoch [5/9], Avg Train Loss: 0.8332, Avg Valid Loss: 1.7330, Valid Accuracy: 0.6625
Epoch [6/9], Avg Train Loss: 0.7557, Avg Valid Loss: 0.8735, Valid Accuracy: 0.7375
Epoch [7/9], Avg Train Loss: 0.7458, Avg Valid Loss: 0.7888, Valid Accuracy: 0.8000
Epoch [8/9], Avg Train Loss: 0.6966, Avg Valid Loss: 0.7855, Valid Accuracy: 0.7625
Epoch [9/9], Avg Train Loss: 0.7073, Avg Valid Loss: 0.7710, Valid Accuracy: 0.7375
Test Accuracy: 0.58



In [ ]:
# Single scale evaluation
S = [256, 384, (256, 512)]
accs = []

for s in S:
    print(f"S = {s}:")
    model = VGG('D', num_classes=class_size).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.0005, momentum=0.9)

    train_loader, valid_loader, test_loader = load_data(batch_size, S=s, 
                                                                      train_dir='./data/flower_photos/train', 
                                                                      val_dir='./data/flower_photos/validation', 
                                                                      test_dir='./data/flower_photos/test')
    
    train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs=10)
    acc = test_model(model, test_loader, device)
    print(f"Test Accuracy: {acc}\n")

In [ ]:
# Multi-scale evaluation

for s in S:
    if isinstance(s, tuple):
        Q = [s[0], (s[0] + s[1])//2, s[1]]
    else:
        Q = [s-32, s, s + 32]
    
    running_acc = 0.

    for q in Q:
        model = VGG('D', num_classes=class_size).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.0005, momentum=0.9)
        train_loader, valid_loader, test_loader = load_data(batch_size, S=s, 
                                                                      train_dir='./data/flower_photos/train', 
                                                                      val_dir='./data/flower_photos/validation', 
                                                                      test_dir='./data/flower_photos/test')
    
        train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs=1)
        acc = test_model(model, test_loader, device)
        running_acc += acc

        del model
        del optimizer
    
    print(f"S={s}, Q={Q}, Avg Accuracy: {running_acc / len(Q)}\n")
